<a href="https://colab.research.google.com/github/srilamaiti/srilamaiti.github.io/blob/main/ml_algo_from_scratch/clustering/kmeans_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np
import random as rd
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
import math
from scipy.spatial.distance import euclidean
import random as rd
import time
rd.seed(42)

In [103]:
class k_means:
    def __init__(self, k = 2, max_iterations = 500, tolerance_level = .00001):
        self.k = k
        self.max_iterations = max_iterations
        self.tolerance_level = tolerance_level

    def distance_between_points(self, point1, point2):
        x1, y1 = point1
        x2, y2 = point2
        return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

    def initialize_centroids(self, data):
        centroids = data[np.random.choice(data.shape[0], self.k, replace=False)]
        return centroids

    def update_centroids(self, closest_centroids):
        points_df = pd.DataFrame(closest_centroids, columns=['point', 'cluster_id'])['point'].apply(pd.Series)
        cluster_ids = pd.DataFrame(closest_centroids).iloc[:, -1].to_frame()  # Convert to DataFrame

        result = pd.concat([points_df, cluster_ids], axis=1)
        result.columns = ['x', 'y', 'cluster']
        return np.array(result.groupby('cluster').agg({'x': 'mean', 'y': 'mean'}))

    def fit(self, X):
        # Initialize centroids randomly
        self.centroids = self.initialize_centroids(X)
        self.closest_centroids = []
        iteration = 0
        while iteration < self.max_iterations:
            for x in X:
                distance = []
                for i, c in enumerate(self.centroids):
                    distance.append((i, self.distance_between_points(x, c)))
                distance.sort(key = lambda x: x[1])
                self.closest_centroids.append((x, distance[0][0]))

            new_centroids = self.update_centroids(self.closest_centroids)
            if np.all(np.abs(self.centroids - new_centroids) < self.tolerance_level):
                break
            self.centroids = new_centroids
            iteration += 1
            self.closest_centroids = []

    def predict(self):
        return pd.DataFrame(self.closest_centroids).iloc[:, -1]


In [83]:
n_samples=10000
k_clusters = 3
max_iterations = 500
tolerance_level = .00001

In [92]:
# Generate synthetic data using sklearn.datasets.make_blobs
X, _ = make_blobs(n_samples=n_samples, centers=k_clusters, random_state=42)
X_sclaed = StandardScaler().fit_transform(X)

In [104]:
custom_kmeans = k_means(k = k_clusters, max_iterations = max_iterations, tolerance_level = tolerance_level)
custom_kmeans.fit(X_sclaed)
labels_custom_kmeans  = custom_kmeans.predict()
labels_custom_kmeans

,1
0,2
1,0
2,2
3,0
4,0
...,...
9995,2
9996,0
9997,0
9998,2


In [105]:
kmeans = KMeans(n_clusters=k_clusters)
kmeans.fit(X_sclaed)
labels  = kmeans.predict(X_sclaed) # The error was due to missing X_sclaed
labels

array([2, 0, 2, ..., 0, 2, 2], dtype=int32)

In [106]:
labels_custom_kmeans == labels

,1
0,True
1,True
2,True
3,True
4,True
...,...
9995,True
9996,True
9997,True
9998,True
